In [12]:
"""Retrieve classAds job information using Spark"""

from __future__ import print_function

from subprocess import Popen, PIPE, STDOUT
from pyspark.sql import Column
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
import pyspark.sql.types as types
import schemas

In [13]:
# Define servers that confirm cache is at UCSD or CalTech
servers = ["xrd-cache-1", "xrd-cache-2", "xcache-00", "xcache-01", "xcache-02", "xcache-03", "xcache-04",
           "xcache-05", "xcache-06", "xcache-07", "xcache-08", "xcache-09", "xcache-10"]
# Get job reports
inputfile = "/project/monitoring/archive/xrootd/raw/gled/2019/06/02/*.json.gz"
jobreports = spark.read.json(inputfile)

In [14]:
# Get dataset
ds = (jobreports
        # Require that server is in defined set
        .filter(col('data.server_host').isin(servers))
        # Require only CMS jobs
        .filter(col('data.vo') == "cms")
        # Select columns to save
        .select(
               col('data.operation').alias('operation'),
               col('data.app_info').alias('app_info'),
               col('data.file_lfn').alias('file_name'),
               col('data.server_host').alias('server_host'),
               col('data.client_host').alias('client_host'),
               col('data.client_domain').alias('client_domain'),
               col('data.start_time').alias('start_time')
               )
    )

In [15]:
# Move from hdfs to eos
p = Popen("hdfs dfs -get /user/jguiang/shared/XRootD_06-02-2019 /eos/user/j/jguiang/data-access/parquet/",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)

print (p.stdout.read())